In [6]:
import json
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [3]:
news = pd.read_json('C:/Users/user/Desktop/pre_삼성전자_연합인포맥스.json')
stock = pd.read_csv('C:/Users/user/Desktop/samsung_stock_price.csv')

In [52]:
def stock_label(news, stock, day):
    
    # Date를 date로 이름 변경 (추후 merge 할때 칼럼명 통일)
    stock.rename(columns = {'Date': 'date'}, inplace = True)
    
    # start_date부터 end_date까지 날짜 리스트 생성
    start_date = datetime.strptime('20010104', '%Y%m%d')
    end_date = datetime.strptime('20201005', '%Y%m%d')

    str_date_list = []
    while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): # start_date가 end_date랑 같아지기 전까지 1씩 늘어나면서 리스트에 추가
        str_date_list.append(start_date.strftime('%Y-%m-%d'))
        start_date += timedelta(days=1)
    str_date_list.append(end_date.strftime('%Y-%m-%d')) # end_date 포함안됐으므로 따로 추가

    date_list=np.array([str_date_list])
    all_date  = pd.DataFrame(date_list.T, columns=['date'])
    
    all_date['date'] = pd.to_datetime(all_date['date'], format="%Y-%m-%d")
    stock['date'] = pd.to_datetime(stock['date'], format="%Y-%m-%d")
    
    df = pd.merge(stock,all_date, on='date', how='outer') # 기존주가데이터와 전체날짜를 date기준으로 outer 조인
    df_stock = df.sort_values(by='date' ,ascending=False).fillna(method='bfill') # 내림차순으로 정렬후, 결측치는 가장 최근 과거 값으로 대입
    df_stock = df_stock.reset_index(drop=True)
    
    # day변수 후의 주가 칼럼 생성
    df_stock['stock_after']=df_stock['Close']
    df_stock.stock_after = df_stock.stock_after.shift(day) # day만큼 shift
    
    #주가 차이 칼럼 생성
    df_stock['price_gap']=''
    df_stock['price_gap'] = df_stock['stock_after']/df_stock['Close']-1 #변화 퍼센트 계산
    
    # 라벨링 칼럼 생성
    df_stock['label']=''
    
    for i in range(len(df_stock)):
        if df_stock['price_gap'][i] > 0:
            df_stock['label'][i] = df_stock['label'][i].replace('', '1')
        elif df_stock['price_gap'][i] < 0:
            df_stock['label'][i] = df_stock['label'][i].replace('', '-1')
        else:
            df_stock['label'][i] = df_stock['label'][i].replace('', '0')
    
    label= df_stock[['date','Close','stock_after','price_gap','label']]
    label['date'] = pd.to_datetime(label['date'], format="%Y-%m-%d")
    label.dropna(axis=0, inplace=True) # 결측치 있는 행 제거
    
    # 뉴스와 결합
    news['date'] = pd.to_datetime(news['date'], format="%Y-%m-%d")
    news_label = pd.merge(news,label, on='date', how='inner')
    
    return news_label

In [53]:
stock_label(news, stock, 5)

<ipython-input-52-5e97801605a5>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock['label'][i] = df_stock['label'][i].replace('', '0')
<ipython-input-52-5e97801605a5>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock['label'][i] = df_stock['label'][i].replace('', '1')
<ipython-input-52-5e97801605a5>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock['label'][i] = df_stock['label'][i].replace('', '-1')
<ipython-

,Unnamed: 0,date,office,url,text,title,count_str,Close,stock_after,price_gap,label
0,477114,2003-08-01,연합인포맥스,https://news.naver.com/main/read.nhn?mode=LSD&...,하이닉스 주가가 급반등했다. 1일 오후 2시23분 현재 하이닉스는 전날보다 1천1...,"하이닉스, 주가 급반등",508,8520.0,8260.0,-0.030516,-1
1,477116,2003-08-01,연합인포맥스,https://news.naver.com/main/read.nhn?mode=LSD&...,거래소시장에서 종합지수가 美 경제지표의 호전에 고무된 외국인 매수로 연중 고점을 ...,"<마감> 거래소,'730선 넘어서자'..연중 고점 경신",1134,8520.0,8260.0,-0.030516,-1
2,477121,2003-08-01,연합인포맥스,https://news.naver.com/main/read.nhn?mode=LSD&...,반도체주들이 강보합권에 개장했다. 1일 오전 9시5분 현재 삼성전자는 전날보다 3...,"반도체주, 반등...미증시 강보합",507,8520.0,8260.0,-0.030516,-1
3,477075,2003-08-01,연합인포맥스,https://news.naver.com/main/read.nhn?mode=LSD&...,1일 전기전자업종은 외국인 매수에 힘입어 삼성전자가 연중 고점을 경신하는 등 대부...,"<전기전자업종> 삼성전자, 연중 최고치 경신",699,8520.0,8260.0,-0.030516,-1
4,477228,2003-08-04,연합인포맥스,https://news.naver.com/main/read.nhn?mode=LSD&...,반도체주들이 약세권에 개장했다. 4일 오전 9시5분 현재 삼성전자는 전날보다 5천...,"반도체주, 하락...하이닉스 보합권 등락",578,8430.0,8120.0,-0.036773,-1
...,...,...,...,...,...,...,...,...,...,...,...
15673,434091,2020-09-28,연합인포맥스,https://news.einfomax.co.kr/news/articleView.h...,코스피는 신종 코로나바이러스 감염증 재확산 우려에도 전일 미국 증시 상승 영향에 ...,"코스피, 美증시 호조에 1%대 상승…2,300선 회복",1103,58200.0,58200.0,0.000000,0
15674,434081,2020-09-28,연합인포맥스,https://news.einfomax.co.kr/news/articleView.h...,코스피는 외국인과 기관의 매수에 오후 상승 폭을 키웠다. 28일 코스피는 오후 ...,"코스피, 1.5%대 상승…外人·기관 매수 지속",856,58200.0,58200.0,0.000000,0
15675,434127,2020-09-29,연합인포맥스,https://news.einfomax.co.kr/news/articleView.h...,코스피는 연휴를 앞두고 기관이 장중 순매수 기조를 이어간 영향으로 상승 마감했다....,"코스피, 0.8%대 상승 마감…기관 사흘째 순매수",986,58200.0,58200.0,0.000000,0
15676,434132,2020-09-29,연합인포맥스,https://news.einfomax.co.kr/news/articleView.h...,코스피는 미국 추가 부양책 합의 기대로 상승하고 있다. 29일 코스피는 9시 6...,"코스피, 개인·기관 매수에 1%대 상승…美 추가 부양책 기대",1073,58200.0,58200.0,0.000000,0
